In [1]:
# from pytorch_fid import fid_score

# dataset_path = 'dataset/training/base'
# generations_path = 'outputs/_saved_examples_50_epochs_base/step5'
# batch_size = 32
# device = 'cuda'
# dims = 64
# # 64: first max pooling features
# # 192: second max pooling features
# # 768: pre-aux classifier features
# # 2048: final average pooling features (this is the default)

# print(fid_score)
# fid_value = fid_score.calculate_fid_given_paths(
#         [dataset_path, generations_path], batch_size, device, dims,
# )

In [2]:
# fid_value

In [3]:
from quality_metrics import nn_metric, ssim_metric, waterstein_distance_metric, fid_metric
from utils.example_generator import generate_examples
from progressive_gan import Generator, Discriminator, train_fn, set_seeds
import os
import torch
import tempfile
import pandas as pd
from math import log2
from tqdm import tqdm

In [4]:
real_datasets = {'2D_assets': './dataset/2D_assets/transformed/', 
                 'bulbapedia': './dataset/bulbapedia/transformed/',
                 'all_assets_and_bulbapedia': './dataset/all_assets_and_bulbapedia/transformed/',
                 'all_assets': './dataset/all_assets/transformed/'
                 }

In [5]:
#256 models have been ignored for fair comparison
generators_paths = [
                    # './outputs/6_test_6_bulbapedia_265x265/generator_128.pth',
                    # './outputs/23_first_try_bulbapedia_23_bulbapedia_256x256_e50/generator_128.pth',
                    # './outputs/29_giga_dataset_29_all_assets_and_bulbapedia_256x256_e50/generator_64_40.pth',
                    # './outputs/31_fine_tune_23_with_giga_dataset_31_all_assets_and_bulbapedia_128x128_e100/generator_128_150.pth',
                    # './outputs/31_fine_tune_23_with_giga_dataset_31_all_assets_and_bulbapedia_128x128_e100/generator_128_130.pth',
                    # './outputs/33_fine_tune_23_with_clean_dataset_33_2D_assets_128x128_e100/generator_128_150.pth',
                    './outputs/1_test_1_2D_assets_128x128_e30/generator_128.pth',
                    './outputs/36_fine_tune_1_with_clean_dataset_36_2D_assets_128x128_e100/generator_128_150.pth'
                    ]

In [6]:
df = pd.DataFrame(columns=['generator', 'dataset', 'ssim', 'waterstein', 'fid', 'path'])

In [7]:
base_path = 'evaluation'
z_dim = 256
in_channels = 256
img_channels = 3
device = 'cuda'
amount_examples = 50
sample = 500

with tqdm(total=len(generators_paths)*len(real_datasets)) as pbar:
    for path in tqdm(generators_paths):
        gen = Generator(
            256, 256, img_channels=img_channels).to(device)
        gen.load_state_dict(torch.load(path))
        generator_size = int(path.split('/')[-1].split('_')[1].split('.')[0])
        step = int(log2(generator_size / 4))
        generator_name = path.split('/')[-2]
        generations_path = os.path.join(base_path, generator_name)
        os.makedirs(generations_path, exist_ok=True)
        generate_examples(gen, step, z_dim, device=device, path=generations_path, n=amount_examples)
        generations_path = os.path.join(generations_path, f'step{step}')
        for ds_name, ds_path in real_datasets.items():
            # nn = nn_metric(ds_path, generations_path)
            ssim = ssim_metric(ds_path, generations_path, sample=sample)
            waterstein = waterstein_distance_metric(ds_path, generations_path, sample=sample)
            fid = fid_metric(ds_path, generations_path, device=device)
            df.loc[len(df)] = [generator_name, ds_name, ssim, waterstein, fid, generations_path]
            # print([generator_name, ds_name, ssim, waterstein, fid, generations_path])
            df.to_csv(os.path.join(base_path,'evaluation.csv'), index=False)
            pbar.update()

  0%|          | 0/8 [00:00<?, ?it/s]

<module 'pytorch_fid.fid_score' from '/root/.local/lib/python3.10/site-packages/pytorch_fid/fid_score.py'>








































































100%|██████████| 69/69 [00:15<00:00,  4.33it/s]


 12%|█▎        | 1/8 [02:03<14:24, 123.47s/it]

<module 'pytorch_fid.fid_score' from '/root/.local/lib/python3.10/site-packages/pytorch_fid/fid_score.py'>

































































































































































100%|██████████| 158/158 [00:31<00:00,  5.09it/s]


 25%|██▌       | 2/8 [04:49<14:50, 148.43s/it]

...
































































































































































100%|██████████| 158/158 [00:34<00:00,  4.54it/s]


 75%|███████▌  | 6/8 [18:10<05:54, 177.41s/it]

<module 'pytorch_fid.fid_score' from '/root/.local/lib/python3.10/site-packages/pytorch_fid/fid_score.py'>







































































































































































































































































































































































100%|██████████| 356/356 [01:13<00:00,  4.86it/s]


 88%|████████▊ | 7/8 [22:58<03:33, 213.83s/it]

<module 'pytorch_fid.fid_score' from '/root/.local/lib/python3.10/site-packages/pytorch_fid/fid_score.py'>













































































































































































































































100%|██████████| 235/235 [00:48<00:00,  4.85it/s]


100%|██████████| 8/8 [26:29<00:00, 198.70s/it]


To compute the FID score between two datasets, where images of each dataset are contained in an individual folder:

`python -m pytorch_fid path/to/dataset1 path/to/dataset2`

To run the evaluation on GPU, use the flag --device cuda:N, where N is the index of the GPU to use.